In [1]:
# Configurar el path para importar los módulos
import sys
import os

# Añadir el directorio raíz del proyecto al path
root_dir = os.chdir(os.path.abspath(os.path.join(os.getcwd(), '..')))
if root_dir not in sys.path:
    sys.path.append(root_dir)

In [2]:
# Importar las funciones necesarias
from db.conexion_db import crear_conexion, cerrar_conexion
from db.querys_db import insert_data, consult_data,procedure_data,procedure_data_param
from abastecimientov2 import reservas_avisorden_ordenes
from datetime import datetime, timedelta
import pandas as pd
import numpy as np

In [3]:
"""
connection = crear_conexion()
procedure_data(connection,"abastece_plan_reservas")
procedure_data(connection,"abastece_plan_avisorden")
procedure_data(connection,"abastece_plan_ordenes")
procedure_data(connection,'abastece_plan_flota')
procedure_data_param(connection, "abastece_plan_pedidospendiente", ('2025-01-01',))   
cerrar_conexion(connection)

"""

'\nconnection = crear_conexion()\nprocedure_data(connection,"abastece_plan_reservas")\nprocedure_data(connection,"abastece_plan_avisorden")\nprocedure_data(connection,"abastece_plan_ordenes")\nprocedure_data(connection,\'abastece_plan_flota\')\nprocedure_data_param(connection, "abastece_plan_pedidospendiente", (\'2025-01-01\',))   \ncerrar_conexion(connection)\n\n'

In [4]:
"""
connection = crear_conexion()
procedure_data(connection,"abastece_plan_inv_mb52")       
cerrar_conexion(connection)     
"""                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

'\nconnection = crear_conexion()\nprocedure_data(connection,"abastece_plan_inv_mb52")       \ncerrar_conexion(connection)     \n'

In [5]:
connection = crear_conexion()
reservas = reservas_avisorden_ordenes(connection)
cerrar_conexion(connection)

Conexion Exitosa
conexion cerrada


In [6]:
connection = crear_conexion()
flota = consult_data(connection,f"select equipo,linegnombre from plan_flota")
cerrar_conexion(connection)

Conexion Exitosa
conexion cerrada


In [7]:
ordenes = pd.read_excel('OC_CGM.xlsx')

In [8]:
ordenes = ordenes[['OC','Material']]

In [9]:
ordenes['llave'] = ordenes['OC'].astype(str) + ordenes['Material'].astype(str)

In [10]:
connection = crear_conexion()
estado_pedidosxpendiente = consult_data(connection,f"select * from plan_pedidospendientes_resumen")
cerrar_conexion(connection)

Conexion Exitosa
conexion cerrada


In [11]:
pl_inventarioMB52 = "plan_inventario_resumen"
connection = crear_conexion()
planinventarioresumen = consult_data(connection,f"select material,centro,total from {pl_inventarioMB52}")
cerrar_conexion(connection)

Conexion Exitosa
conexion cerrada


In [12]:
tbl_inventario = pd.DataFrame(planinventarioresumen)
tbl_flota = pd.DataFrame(flota)
tbl_estado_pedidosxpendiente = pd.DataFrame(estado_pedidosxpendiente)

In [13]:
tbl_estado_pedidosxpendiente['llave'] = tbl_estado_pedidosxpendiente['doc_compra'].astype(str) + tbl_estado_pedidosxpendiente['material'].astype(str)

In [14]:
materiales = tbl_estado_pedidosxpendiente.loc[(tbl_estado_pedidosxpendiente['llave'].isin(ordenes['llave']))]

In [15]:
materiales = materiales[['material','centro','por_entr_ctd']]

In [16]:
materiales.rename(columns={'por_entr_ctd': 'total'}, inplace=True)

In [17]:
tbl_inventario_xmateriales = pd.concat([tbl_inventario, materiales], axis=0) 

In [18]:
tbl_inventario_resumen = tbl_inventario_xmateriales.groupby(['material','centro']).sum().reset_index()

In [19]:
tbl_estado_pedidosxpendiente = tbl_estado_pedidosxpendiente.loc[~(tbl_estado_pedidosxpendiente['llave'].isin(ordenes['llave']))]

In [20]:

tbl_estado_pedidosxpendiente['identificador'] = tbl_estado_pedidosxpendiente['doc_compra'] + tbl_estado_pedidosxpendiente['pos']

In [21]:
filtrar_cabezeras_estadoxpedidos = tbl_estado_pedidosxpendiente[['identificador','doc_compra','material','centro','por_entr_ctd','fecha_entrega','nombre_proveedor']].copy()
tbl_inventario_resumen['total'] = tbl_inventario_resumen['total'].astype(float)
filtrar_cabezeras_estadoxpedidos['por_entr_ctd'] = filtrar_cabezeras_estadoxpedidos['por_entr_ctd'].astype(float)
filtrar_cabezeras_estadoxpedidos['fecha_entrega'] = pd.to_datetime(filtrar_cabezeras_estadoxpedidos['fecha_entrega'])
filtrar_cabezeras_estadoxpedidos['nombre_proveedor'] = filtrar_cabezeras_estadoxpedidos['nombre_proveedor'].str[0:10]

In [22]:
reservasCxC = reservas.loc[(reservas['ate_accion']=='MERCADERIA') & (reservas['ctd_dif'] > 0) ]
filtrar_cabezeras_reservasCxC = reservasCxC[['index','identificador','material','centro_x','ctd_dif','stock_asignado','trasl_ctd_C154','trasl_ctd_C040',
                                             'trasl_ctd_C080','trasl_ctd_C200','trasl_ctd_C152','trasl_fecha','Ped_nro','Ped_Fecha','Ped_ctd','Ped_proveedor']].copy()

In [23]:
filtrar_cabezeras_estadoxpedidos.columns = filtrar_cabezeras_estadoxpedidos.columns.str.lower()
filtrar_cabezeras_reservasCxC.rename(columns={'centro_x' : 'centro'},inplace=True)

In [24]:
filtrar_cabezeras_estadoxpedidos.loc[:,'material'] = filtrar_cabezeras_estadoxpedidos.loc[:,'material'].str.replace(' ','')
tbl_inventario_resumen.loc[:,'material'] = tbl_inventario_resumen.loc[:,'material'].str.replace(' ','')
filtrar_cabezeras_reservasCxC.loc[:,'material'] = filtrar_cabezeras_reservasCxC.loc[:,'material'].str.replace(' ','')

In [25]:
tbl_inventario_resumen['identificador'] = tbl_inventario_resumen['material'] + tbl_inventario_resumen['centro']

In [26]:
def _asign_stock_centros(reservas_grupo, inventario_grupo):
    # Obtener el stock disponible para el grupo
    stock_disponible = inventario_grupo['total'].iloc[0]
    
    # Inicializar la cantidad asignada
    cantidad_asignada = []
    stock_restante = stock_disponible
    
    # Calcular la cantidad asignada para cada reserva
    for ctd_dif in reservas_grupo['ctd_dif']:
        if stock_restante <= 0:
            # Si no hay stock disponible, asignar 0
            cantidad_asignada.append(0.0)
        else:
            # Asignar la cantidad mínima entre lo pedido y el stock restante
            asignacion = min(ctd_dif, stock_restante)
            cantidad_asignada.append(asignacion)
            stock_restante -= asignacion
    
    # Asignar la cantidad ajustada usando .loc para evitar SettingWithCopyWarning
    reservas_grupo.loc[:, 'stock_asignado'] = cantidad_asignada
    
    # Actualizar el inventario restante
    inventario_grupo.loc[:, 'total'] = stock_restante
    
    return reservas_grupo, inventario_grupo

In [27]:
def asignar_stock_centroxcentro(reservas, inventario):
    # Asegurarse de que las reservas estén ordenadas por la columna 'index'
    reservas = reservas.sort_values(by='index')
    
    # Listas para almacenar resultados
    reservas_actualizadas = []
    inventario_actualizado = []
    
    # Agrupar por 'centro' y 'material'
    grupos = reservas.groupby(['centro', 'material'])
    
    for (centro, material), reservas_grupo in grupos:
        # Filtrar el inventario para el grupo actual
        inventario_grupo = inventario[(inventario['centro'] == centro) & (inventario['material'] == material)]
        
        if not inventario_grupo.empty:
            # Procesar el grupo
            reservas_procesadas, inventario_procesado = _asign_stock_centros(reservas_grupo, inventario_grupo)
            reservas_actualizadas.append(reservas_procesadas)
            inventario_actualizado.append(inventario_procesado)
        else:
            # Si no hay inventario, mantener las reservas sin cambios
            reservas_actualizadas.append(reservas_grupo)
    
    # Concatenar resultados
    reservas_final = pd.concat(reservas_actualizadas, ignore_index=True)
    inventario_final = pd.concat(inventario_actualizado, ignore_index=True)
    
    return reservas_final, inventario_final

In [28]:
pedido_modif, inv_modif = asignar_stock_centroxcentro(filtrar_cabezeras_reservasCxC,tbl_inventario_resumen)

In [29]:
obtener_stockasignado = pedido_modif[['identificador','stock_asignado']]
obtener_stock_modificado = inv_modif[['identificador','total']]

In [30]:
modificar_stockasignado_reservas = filtrar_cabezeras_reservasCxC.merge(obtener_stockasignado,how='left',on='identificador')
modificar_stock_inventario = tbl_inventario_resumen.merge(obtener_stock_modificado,how='left',on='identificador')

In [31]:
modificar_stockasignado_reservas.drop(columns='stock_asignado_x', inplace=True)
modificar_stockasignado_reservas.rename(columns={'stock_asignado_y' : 'stock_asignado'}, inplace=True)

In [32]:
modificar_stockasignado_reservas['diferencia'] = modificar_stockasignado_reservas['ctd_dif'] - modificar_stockasignado_reservas['stock_asignado']
reservas_pendientes = modificar_stockasignado_reservas.loc[modificar_stockasignado_reservas['diferencia'] >0].copy()

reservas_pendientes.drop(columns='diferencia', inplace=True)
modificar_stockasignado_reservas.drop(columns='diferencia', inplace=True)


In [33]:
reservas_pendientes['total'] = reservas_pendientes['stock_asignado']
modificar_stockasignado_reservas['total'] = modificar_stockasignado_reservas['stock_asignado']

In [34]:
modificar_stock_inventario.loc[modificar_stock_inventario['total_y'].isna(), 'total_y'] = modificar_stock_inventario['total_x']
modificar_stock_inventario['total'] = modificar_stock_inventario['total_y']
modificar_stock_inventario.drop(columns={'total_x','total_y'}, inplace= True)

In [35]:
stock_restante = modificar_stock_inventario.loc[modificar_stock_inventario['total'] > 0].copy()

In [36]:
def _asign_pedidos_compra(reservas, pedidospendiente):
    # Ordenar los pedidos pendientes
    ped = pedidospendiente.sort_values(by=['fecha_entrega', 'identificador'], ascending=[True, True], na_position='last')

    # Proceso de asignación
    for i, reser in reservas.iterrows():
        cantidad_pedida = reser['ctd_dif'] - reser['total']
        compras_consolidadas = []
        fechas_consolidadas = []
        cantidades_consolidadas = []
        proveedor_consolidadas = []

        for j, pedi in ped.iterrows():
            if pedi['por_entr_ctd'] > 0:
                if pedi['por_entr_ctd'] >= cantidad_pedida:
                    compras_consolidadas.append(pedi['doc_compra'])
                    fechas_consolidadas.append(pedi['fecha_entrega'])
                    proveedor_consolidadas.append(pedi['nombre_proveedor'])
                    cantidades_consolidadas.append(cantidad_pedida)
                    ped.at[j, 'por_entr_ctd'] -= cantidad_pedida
                    cantidad_pedida = 0
                    break
                else:
                    compras_consolidadas.append(pedi['doc_compra'])
                    fechas_consolidadas.append(pedi['fecha_entrega'])
                    proveedor_consolidadas.append(pedi['nombre_proveedor'])
                    cantidades_consolidadas.append(pedi['por_entr_ctd'])
                    cantidad_pedida -= pedi['por_entr_ctd']
                    ped.at[j, 'por_entr_ctd'] = 0

        reservas.at[i, 'Ped_nro'] = ', '.join(map(str, compras_consolidadas))
        reservas.at[i, 'Ped_ctd'] = sum(cantidades_consolidadas)
        reservas.at[i, 'Ped_proveedor'] = ', '.join(map(str, proveedor_consolidadas))
        reservas.at[i, 'total'] = reser['total'] + sum(cantidades_consolidadas)
        if fechas_consolidadas:
            reservas.at[i, 'Ped_Fecha'] = fechas_consolidadas[-1]

    return reservas, ped

In [37]:
def asignacion_pedidoscompra(reservas, pedidoscompras):
    reserva = reservas.copy()
    pedidoscompra = pedidoscompras.copy()

    centros = pedidoscompra['centro'].unique()
    materiales = pedidoscompra['material'].unique()

    concatenar_reservas = []

    for centro in centros:
        for material in materiales:
            pedido_m = reserva[(reserva['centro'] == centro) & (reserva['material'] == material)]
            pedidos_filtrados = pedidoscompra[(pedidoscompra['centro'] == centro) & (pedidoscompra['material'] == material)]
            if not pedidos_filtrados.empty:
                _reservas, _pedidos = _asign_pedidos_compra(pedido_m, pedidos_filtrados)
                concatenar_reservas.append(_reservas)
                pedidoscompra.update(_pedidos)

    pedidos_consolidados = pd.concat(concatenar_reservas)

    return pedidos_consolidados, pedidoscompra

In [38]:
pedido_modif7,pedidoscompra_modif = asignacion_pedidoscompra(reservas_pendientes, filtrar_cabezeras_estadoxpedidos)

In [39]:
fil_pedido_modif7 = pedido_modif7[['identificador','Ped_nro','Ped_Fecha','Ped_ctd','Ped_proveedor','total']]
fil_pedidoscompra =pedidoscompra_modif.loc[pedidoscompra_modif['por_entr_ctd'] >0]

In [40]:
agregar_pedidosxreservas = modificar_stockasignado_reservas.merge(fil_pedido_modif7,how='left', on='identificador')

In [41]:
agregar_pedidosxreservas.loc[agregar_pedidosxreservas['Ped_nro_y'].isna(), 'Ped_nro_y'] = agregar_pedidosxreservas['Ped_nro_x']
agregar_pedidosxreservas.loc[agregar_pedidosxreservas['Ped_Fecha_y'].isna(), 'Ped_Fecha_y'] = agregar_pedidosxreservas['Ped_Fecha_x']
agregar_pedidosxreservas.loc[agregar_pedidosxreservas['Ped_ctd_y'].isna(), 'Ped_ctd_y'] = agregar_pedidosxreservas['Ped_ctd_x']
agregar_pedidosxreservas.loc[agregar_pedidosxreservas['Ped_proveedor_y'].isna(), 'Ped_proveedor_y'] = agregar_pedidosxreservas['Ped_proveedor_x']
agregar_pedidosxreservas.loc[agregar_pedidosxreservas['total_y'].isna(), 'total_y'] = agregar_pedidosxreservas['total_x']

In [42]:
agregar_pedidosxreservas['Ped_nro'] = agregar_pedidosxreservas['Ped_nro_y']
agregar_pedidosxreservas['Ped_Fecha'] = agregar_pedidosxreservas['Ped_Fecha_y']
agregar_pedidosxreservas['Ped_ctd'] = agregar_pedidosxreservas['Ped_ctd_y']
agregar_pedidosxreservas['Ped_proveedor'] = agregar_pedidosxreservas['Ped_proveedor_y']
agregar_pedidosxreservas['total'] = agregar_pedidosxreservas['total_y']

agregar_pedidosxreservas.drop(columns=['Ped_nro_y','Ped_nro_x'], inplace=True)
agregar_pedidosxreservas.drop(columns=['Ped_Fecha_y','Ped_Fecha_x'], inplace=True)
agregar_pedidosxreservas.drop(columns=['Ped_ctd_y','Ped_ctd_x'], inplace=True)
agregar_pedidosxreservas.drop(columns=['Ped_proveedor_y','Ped_proveedor_x'], inplace=True)
agregar_pedidosxreservas.drop(columns=['total_y','total_x'], inplace=True)


In [43]:
fil_reserv = agregar_pedidosxreservas[['identificador','stock_asignado','trasl_ctd_C154','trasl_ctd_C152','trasl_ctd_C200','trasl_ctd_C040','trasl_ctd_C080','Ped_nro','Ped_Fecha','Ped_ctd','Ped_proveedor','total']]

In [44]:
stock_restante_pivot = stock_restante.pivot_table(index='material',columns='centro', values='total', aggfunc='sum').reset_index()

In [45]:
abas_reservas = reservas.merge(fil_reserv, how='left', on='identificador')

In [46]:
abastecimiento_reservas = abas_reservas.merge(stock_restante_pivot, how='left', on='material')

In [47]:
abastecimiento_reservas['total_trasl'] = 0

In [48]:
abastecimiento_reservas.loc[abastecimiento_reservas['stock_asignado_y'].isna(), 'stock_asignado_y'] = abastecimiento_reservas['stock_asignado_x']
abastecimiento_reservas.loc[abastecimiento_reservas['trasl_ctd_C154_y'].isna(), 'trasl_ctd_C154_y'] = abastecimiento_reservas['trasl_ctd_C154_x']
abastecimiento_reservas.loc[abastecimiento_reservas['trasl_ctd_C152_y'].isna(), 'trasl_ctd_C152_y'] = abastecimiento_reservas['trasl_ctd_C152_x']
abastecimiento_reservas.loc[abastecimiento_reservas['trasl_ctd_C200_y'].isna(), 'trasl_ctd_C200_y'] = abastecimiento_reservas['trasl_ctd_C200_x']
abastecimiento_reservas.loc[abastecimiento_reservas['trasl_ctd_C040_y'].isna(), 'trasl_ctd_C040_y'] = abastecimiento_reservas['trasl_ctd_C040_x']
abastecimiento_reservas.loc[abastecimiento_reservas['trasl_ctd_C080_y'].isna(), 'trasl_ctd_C080_y'] = abastecimiento_reservas['trasl_ctd_C080_x']
abastecimiento_reservas.loc[abastecimiento_reservas['Ped_nro_y'].isna(), 'Ped_nro_y'] = abastecimiento_reservas['Ped_nro_x']
abastecimiento_reservas.loc[abastecimiento_reservas['Ped_Fecha_y'].isna(), 'Ped_Fecha_y'] = abastecimiento_reservas['Ped_Fecha_x']
abastecimiento_reservas.loc[abastecimiento_reservas['Ped_ctd_y'].isna(), 'Ped_ctd_y'] = abastecimiento_reservas['Ped_ctd_x']
abastecimiento_reservas.loc[abastecimiento_reservas['Ped_proveedor_y'].isna(), 'Ped_proveedor_y'] = abastecimiento_reservas['Ped_proveedor_x']
abastecimiento_reservas.loc[abastecimiento_reservas['total'].isna(), 'total'] = 0
abastecimiento_reservas.loc[abastecimiento_reservas['total_trasl'].isna(), 'total_trasl'] = 0


In [49]:
abastecimiento_reservas['stock_asignado'] = abastecimiento_reservas['stock_asignado_y']
abastecimiento_reservas['trasl_ctd_C154'] = abastecimiento_reservas['trasl_ctd_C154_y']
abastecimiento_reservas['trasl_ctd_C152'] = abastecimiento_reservas['trasl_ctd_C152_y']
abastecimiento_reservas['trasl_ctd_C200'] = abastecimiento_reservas['trasl_ctd_C200_y']
abastecimiento_reservas['trasl_ctd_C040'] = abastecimiento_reservas['trasl_ctd_C040_y']
abastecimiento_reservas['trasl_ctd_C080'] = abastecimiento_reservas['trasl_ctd_C080_y']

abastecimiento_reservas['Ped_nro'] = abastecimiento_reservas['Ped_nro_y']
abastecimiento_reservas['Ped_Fecha'] = abastecimiento_reservas['Ped_Fecha_y']
abastecimiento_reservas['Ped_ctd'] = abastecimiento_reservas['Ped_ctd_y']
abastecimiento_reservas['Ped_proveedor'] = abastecimiento_reservas['Ped_proveedor_y']

abastecimiento_reservas.drop(columns=['stock_asignado_y','stock_asignado_x'], inplace=True)
abastecimiento_reservas.drop(columns=['trasl_ctd_C154_y','trasl_ctd_C154_x'], inplace=True)
abastecimiento_reservas.drop(columns=['trasl_ctd_C152_y','trasl_ctd_C152_x'], inplace=True)
abastecimiento_reservas.drop(columns=['trasl_ctd_C200_y','trasl_ctd_C200_x'], inplace=True)
abastecimiento_reservas.drop(columns=['trasl_ctd_C040_y','trasl_ctd_C040_x'], inplace=True)
abastecimiento_reservas.drop(columns=['trasl_ctd_C080_y','trasl_ctd_C080_x'], inplace=True)

abastecimiento_reservas.drop(columns=['Ped_nro_y','Ped_nro_x'], inplace=True)
abastecimiento_reservas.drop(columns=['Ped_Fecha_y','Ped_Fecha_x'], inplace=True)
abastecimiento_reservas.drop(columns=['Ped_ctd_y','Ped_ctd_x'], inplace=True)
abastecimiento_reservas.drop(columns=['Ped_proveedor_y','Ped_proveedor_x'], inplace=True)

In [50]:
abastecimiento_reservas['trasl_ctd_C154'] = abastecimiento_reservas['C154']
abastecimiento_reservas['trasl_ctd_C152'] = abastecimiento_reservas['C152']
abastecimiento_reservas['trasl_ctd_C200'] = abastecimiento_reservas['C200']
abastecimiento_reservas['trasl_ctd_C040'] = abastecimiento_reservas['C040']
abastecimiento_reservas['trasl_ctd_C080'] = abastecimiento_reservas['C080']

In [51]:
abastecimiento_reservas.drop(columns=['C154'], inplace=True)
abastecimiento_reservas.drop(columns=['C152'], inplace=True)
abastecimiento_reservas.drop(columns=['C200'], inplace=True)
abastecimiento_reservas.drop(columns=['C040'], inplace=True)
abastecimiento_reservas.drop(columns=['C080'], inplace=True)


In [52]:
abastecimiento_reservas['Q_pendiente'] = abastecimiento_reservas['ctd_dif'] - abastecimiento_reservas['total'] 

In [53]:
abastecimiento_reservas['Est_material'] = ''
abastecimiento_reservas['Est_abastecimiento'] = ''
abastecimiento_reservas.loc[abastecimiento_reservas['Q_pendiente'] == 0,'Est_material'] = 'Completo'
abastecimiento_reservas.loc[abastecimiento_reservas['Q_pendiente'] > 0,'Est_material'] = 'Faltante'
abastecimiento_reservas.loc[(abastecimiento_reservas['Q_pendiente'] > 0) & ((abastecimiento_reservas['ctd_dif'] - abastecimiento_reservas['Q_pendiente']) != 0),'Est_material'] = 'Parcial'

abastecimiento_reservas.loc[abastecimiento_reservas['Q_pendiente'] == 0,'Est_abastecimiento'] = 'Abastecido'
abastecimiento_reservas.loc[abastecimiento_reservas['Q_pendiente'] != 0,'Est_abastecimiento'] = 'No_Abastecido'

In [54]:
abastecimiento_reservas['Est_Compra'] = ''
abastecimiento_reservas.loc[((abastecimiento_reservas['ctd_dif'] - abastecimiento_reservas['stock_asignado']) ==0),'Est_Compra'] = 'Total_abs_inv'
abastecimiento_reservas.loc[( abastecimiento_reservas['Ped_ctd'] >0),'Est_Compra'] = 'Segmien_Oc'
abastecimiento_reservas.loc[abastecimiento_reservas['Q_pendiente'] >0,'Est_Compra'] = 'Por_Pedir'

In [55]:
abastecimiento_reservas['tipo_Ot'] = abastecimiento_reservas['orden_texto'].str[:3]
abastecimiento_reservas['semana_Ot_txt'] = abastecimiento_reservas['orden_revision_texto'].str[-2:]

abastecimiento_reservas['fe_nece'] = pd.to_datetime(abastecimiento_reservas['fe_nece'])
abastecimiento_reservas['felib'] = pd.to_datetime(abastecimiento_reservas['felib'])

abastecimiento_reservas['semana_Ot'] = abastecimiento_reservas['fe_nece'].dt.isocalendar().week
abastecimiento_reservas['semana_lib'] = abastecimiento_reservas['felib'].dt.isocalendar().week


In [56]:
abast_reservas = abastecimiento_reservas.merge(tbl_flota, how='left', on='equipo')

In [57]:
abast_reservas['fechahora'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')


In [58]:
cargar_abastecimiento = abast_reservas.reindex(['index','fechahora','identificador','orden','felib','aviso','empldeno','zona','equipo','linegnombre','modelo','eqdescrip','repercusion','cliente','proyecto','ave_ini_fecha','ave_fin_fecha','parada',
                                        'parada_dias','ubitec_deno','local','undneg','centro_deno','eq_est_usu_deno','flota_tipo','fe_despacho','eq_serie_motor','eq_serie_equipo','eq_est_ot_deno','clase','grupo_planif',
                                        'cliente_nro','proyecto_nro','supres_nombre','criticidad_deno','eqvaloradq_usd','repercusiondeno','oden_clase_deno','coti_esta_deno','avisodespacho','estatususuarioorden',
                                        'aviso_fecreado','orden_fe_fin_ext','orden_fe_ini_ext','fabricaequipo','orden_fecietec','orden_claact_deno','gruplan_deno','centro_y','eq_clase','orden_tipimput','orden_fecrea','orden_texto',
                                        'orden_prio','prio_orden','prio_orden_texto','ot_abierto','ot_liberado','ot_cierrete','ot_cerrado','hruta_contador','hruta_grupo','aviso_prioridad_deno','aviso_clase_deno','orden_revision','orden_revision_texto','orden_causas_cod',
                                        'orden_causas','orden_causas_texto','cotizacion','pedido_venta','orden_pos_mtto','plan_mantenimiento','fechacarga_y','reserva','reservapos','material','materialdeno','ctd_nec','ctd_reduc',
                                        'ctd_dif','um_base','usuario','centro_x','almacen','lote','fe_nece','orden_op','dest_merc','imputacion','ce_coste','salida_fin','relevplnec','grafo','elem_pep','febase','reservaestatus','reservaimputacion',
                                        'indicador_dh','reserva_pos_borrado','reserva_mov_permit','ate_accion','ate_codigosap','ate_cantidad','ate_UMB','ate_orden','ate_fecha','valor_flota','reservasolped','detalle01','detalle02',
                                        'stock_asignado','trasl_ctd_C154','trasl_ctd_C040','trasl_ctd_C080','trasl_ctd_C200','trasl_ctd_C152','trasl_fecha','Ped_nro','Ped_Fecha','Ped_ctd','Ped_proveedor','estadoreserva','Cod_mate_antiguo','Grp_mat','Jerarquia_produc','total_trasl',
                                        'Q_pendiente','total','Est_material','Est_abastecimiento','Est_Compra','tipo_Ot','semana_Ot_txt','semana_Ot','semana_lib'],axis=1)

In [59]:
cabezera_abastecimiento = ['id','fechacarga','identificador','orden','felib','aviso','empldeno','zona','equipo','linea_negocio','modelo','eqdescrip','repercusion','cliente','proyecto','ave_ini_fecha','ave_fin_fecha','parada','parada_dias','ubitec_deno',
                            'local','undneg','centro_deno','eq_est_usu_deno','flota_tipo','fe_despacho','eq_serie_motor','eq_serie_equipo','eq_est_ot_deno','clase','grupo_planif','cliente_nro','proyecto_nro','supres_nombre',
                            'criticidad_deno','eqvaloradq_usd','repercusiondeno','oden_clase_deno','coti_esta_deno','avisodespacho','estatususuarioorden','aviso_fecreado','orden_fe_fin_ext','orden_fe_ini_ext','fabricaequipo',
                            'orden_fecietec','orden_claact_deno','gruplan_deno','centro','eq_clase','orden_tipimput','orden_fecrea','orden_texto','orden_prio','prio_orden','prio_orden_texto','ot_abierto','ot_liberado','ot_cierrete','ot_cerrado','hruta_contador',
                            'hruta_grupo','aviso_prioridad_deno','aviso_clase_deno','orden_revision','orden_revision_texto','orden_causas_cod','orden_causas','orden_causas_texto','cotizacion','pedido_venta','orden_pos_mtto','plan_mantenimiento',
                            'reserva_fechacarga','reserva','reservapos','material','materialdeno','ctd_nec','ctd_reduc','ctd_dif','um_base','usuario','reserva_centro','reserva_almacen','lote','fe_nece','orden_op','dest_merc','imputacion','ce_coste',
                            'salida_fin','relevplnec','grafo','elem_pep','febase','reservaestatus','reservaimputacion','indicador_dh','reserva_pos_borrado','reserva_mov_permit','ate_accion','ate_codigosap','ate_cantidad','ate_UMB','ate_orden',
                            'ate_fecha','valor_flota','reservasolped','detalle01','detalle02','stock_asignado','trasl_ctd_c154','trasl_ctd_c040','trasl_ctd_c080','trasl_ctd_c200','trasl_ctd_c152','trasl_fecha','Ped_nro','Ped_Fecha','Ped_ctd','Ped_proveedor','estadoreserva',
                            'Cod_mat_antig','Grp_material','Jerarquia','total_trasl','Q_pendiente','Q_asignado','Est_material','Est_abastecimiento','Estado_Compra','tipo_Ot','semana_Ot_txt','semana_Ot','semana_lib', "dias_en_estado_actual", "comentarios"]

In [60]:
cargar_abastecimientosin_diesel = cargar_abastecimiento[cargar_abastecimiento['material'] != 'DIESEL'].copy()

In [69]:
connection = crear_conexion()
abastecimiento_v3 = consult_data(connection,f"select identificador,fechacarga,Estado_Compra,dias_en_estado_actual from tbl_abastecimiento_v3")
cerrar_conexion(connection)

Conexion Exitosa
conexion cerrada


In [71]:
connection = crear_conexion()
comentarios = consult_data(connection,f"select key_unica,comentarios from tbl_seguimiento_compras_VPSCIRO")
cerrar_conexion(connection)

Conexion Exitosa
conexion cerrada


In [139]:
df_abastecimiento_v3 = pd.DataFrame(abastecimiento_v3)
df_comentarios = pd.DataFrame(comentarios).rename(columns={'key_unica': 'identificador'})

In [122]:
fecha_anterior = df_abastecimiento_v3['fechacarga'].dt.strftime('%Y-%m-%d').unique()

In [123]:
fecha_actual = datetime.now().strftime('%Y-%m-%d')

In [129]:
def fecha_cambiada(cargar_abastecimiento,abastecimiento_anterior ):
    abastecimiento_actual = cargar_abastecimiento[["identificador","Est_Compra"]].copy()
    abastecimiento_anterior = abastecimiento_anterior[["identificador","Estado_Compra","dias_en_estado_actual"]].copy()

    #prueba = abastecimiento_actual.loc[abastecimiento_actual['identificador'] == 'P73998900126468']
    #prueba1 = abastecimiento_anterior.loc[abastecimiento_anterior['identificador'] == 'P73998900126468']
    #print(prueba)
    #print(prueba1)
    abastecimiento_merged = abastecimiento_actual.merge(
    abastecimiento_anterior,
    on='identificador',
    how='left')

    # 2. Aplicar la lógica de actualización de 'dias_en_estado_actual'
    abastecimiento_merged['dias_en_estado_actual'] = abastecimiento_merged.apply(
        lambda row: 0 if row['Est_Compra'] != row['Estado_Compra']
        else (row['dias_en_estado_actual'] + 1 if pd.notnull(row['dias_en_estado_actual']) else 0),
        axis=1
    )

    # 3. Quedarte solo con las columnas necesarias, con nombres limpios
    abastecimiento_actualizado = abastecimiento_merged[[
        'identificador',
        'dias_en_estado_actual'
    ]]

    return abastecimiento_actualizado

In [130]:
def dentro_fecha(cargar_abastecimiento,abastecimiento_anterior ):
    abastecimiento_actual = cargar_abastecimiento[["identificador","Est_Compra"]].copy()
    abastecimiento_anterior = abastecimiento_anterior[["identificador","Estado_Compra","dias_en_estado_actual"]].copy()

    #prueba = abastecimiento_actual.loc[abastecimiento_actual['identificador'] == 'P73998900126468']
    #prueba1 = abastecimiento_anterior.loc[abastecimiento_anterior['identificador'] == 'P73998900126468']
    #print(prueba)
    #print(prueba1)
    abastecimiento_merged = abastecimiento_actual.merge(
    abastecimiento_anterior,
    on='identificador',
    how='left')

    # 2. Aplicar la lógica de actualización de 'dias_en_estado_actual'
    abastecimiento_merged['dias_en_estado_actual'] = abastecimiento_merged.apply(
        lambda row: 0 if row['Est_Compra'] != row['Estado_Compra']
        else (row['dias_en_estado_actual'] if pd.notnull(row['dias_en_estado_actual']) else 0),
        axis=1
    )

    # 3. Quedarte solo con las columnas necesarias, con nombres limpios
    abastecimiento_actualizado = abastecimiento_merged[[
        'identificador',
        'dias_en_estado_actual'
    ]]

    return abastecimiento_actualizado

In [131]:
if fecha_anterior < fecha_actual:

    abastecimiento_actualizado = fecha_cambiada(cargar_abastecimientosin_diesel, df_abastecimiento_v3)
elif fecha_anterior == fecha_actual:

    abastecimiento_actualizado = dentro_fecha(cargar_abastecimientosin_diesel, df_abastecimiento_v3)
else:
    print("Revisar la fecha de mysql")
    

In [135]:
cargaxdias = cargar_abastecimientosin_diesel.merge(abastecimiento_actualizado, how='left', on='identificador')

In [140]:
abas_diasxcommet = cargaxdias.merge(df_comentarios, how='left', on='identificador')

In [ ]:
connection = crear_conexion()
consult_data(connection,"truncate table tbl_abastecimiento_v3")
insert_data(connection,'tbl_abastecimiento_v3',abas_diasxcommet,cabezera_abastecimiento,1000)

Conexion Exitosa
Se insertaron: 6758 registros. en la tabla tbl_abastecimiento_v3


In [62]:
len(cargar_abastecimiento.loc[cargar_abastecimiento['ate_accion']=='FIN'])

297